In [2]:
using Pkg
Pkg.add("ProgressBars")

   Updating registry at `~/.julia/registries/General`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`


In [3]:
using Random, LinearAlgebra, Plots
# bring packages into main namespace
using DataFrames             # Data tables are called "DataFrames"
using StatsPlots             # load plotting packages 
using Statistics             # basic statistical functions
using CSV                    # tools for working with CSV files
using Plots, Random, LinearAlgebra, Statistics, SparseArrays
using ProgressBars
include("proxgrad.jl")
pyplot()

Plots.PyPlotBackend()

In [5]:
dataset = CSV.read("Consumer_Airfare_Report__Table_1a_-_All_U.S._Airport_Pair_Markets.csv")
dataset = dropmissing(dataset)

,tbl,Year,quarter,citymarketid_1,citymarketid_2,city1
,String,Int64,Int64,Int64,Int64,String
1,Table 1a,2010,1,34614,33195,"Salt Lake City, UT"
2,Table 1a,1998,4,30189,31703,"Colorado Springs, CO"
3,Table 1a,1998,4,30198,30852,"Pittsburgh, PA"
4,Table 1a,2009,3,32211,32575,"Las Vegas, NV"
5,Table 1a,1993,4,30255,30852,"Huntsville, AL"
6,Table 1a,2010,4,33198,32575,"Kansas City, MO"
7,Table 1a,2007,2,30977,33495,"Chicago, IL"
8,Table 1a,2003,2,30257,30977,"Albany, NY"
9,Table 1a,2005,4,30721,30792,"Boston, MA (Metropolitan Area)"


In [49]:
dataset[:when] = dataset[:Year] .+ ((dataset[:quarter] .-1) ./ 4)
n = size(dataset,1)
r = .1
cuttoff = Int(round(n*r))
idxs = rand(1:n,n)

# For Extrapolation
cuttoff_test = 2015
cuttoff_val = 2011.25

val_splitter(t) = t >= cuttoff_val && t < cuttoff_test

train_idxs = dataset[:when].< cuttoff_val
val_idxs  = val_splitter.(dataset[:when])
test_idxs = dataset[:when].>= cuttoff_test

train_dataset = dataset[train_idxs,:]
val_dataset  = dataset[val_idxs,:]
test_dataset  = dataset[test_idxs,:]
    
n_train = size(train_dataset,1)
n_val = size(val_dataset,1)
n_test = size(test_dataset,1)
@assert  n_train + n_val + n_test == n
@show n_train, n_val, n_test
@show n_train / n, n_val / n, n_test / n


(n_train, n_val, n_test) = (139910, 30802, 30680)
(n_train / n, n_val / n, n_test / n) = (0.6947147850957337, 0.15294549932470008, 0.15233971557956622)


(0.6947147850957337, 0.15294549932470008, 0.15233971557956622)

In [60]:
# Do k-fold cross validation and return the average error_metric on the validation set accross the k folds.
function cross_val(featurizer, loss, regularizer, stepsize, error_metric; k=10, dataset=train_dataset)
    X,y = featurizer(dataset)
    n = size(dataset,1)
    r = Int(round(n / k))
    idxs = rand(1:n,n) # to shuffle the dataset
    error = 0
    for i in (1:k)
        val_idxs = r*(i-1)+1:min(r*i,n)
        tr_low = 1:r*(i-1)
        tr_high = r*i+1:n
        if (i == 1)
            tr_idxs = tr_high
        elseif (i == k)
            tr_idxs = tr_low
        else
            tr_idxs = [tr_low ; tr_high ]
        end
        # @show i, val_idxs
        # @show tr_low, tr_high
        X_tr = X[idxs[tr_idxs],:]
        y_tr = y[idxs[tr_idxs]]
        
        X_val = X[idxs[val_idxs],:]
        y_val = y[idxs[val_idxs]]
        
        w = proxgrad(loss, regularizer, X_tr, y_tr; stepsize=stepsize) 
        ŷ_val = X_val * w
        # @show size(y_val)
        # @show size(ŷ_val)
        error += error_metric(ŷ_val, y_val)
    end
    return error / k
end

function extrapolation_val(featurizer, loss, regularizer, stepsize, error_metric)
    X_tr,y_tr = featurizer(train_dataset)
    X_val,y_val = featurizer(val_dataset)

    w = proxgrad(loss, regularizer, X_tr, y_tr; stepsize=stepsize) 
    ŷ_val = X_val * w
    # @show size(y_val)
    # @show size(ŷ_val)
    
    return error_metric(ŷ_val, y_val)
end

function train_and_validate(featurizer, loss, regularizer, stepsize, error_metric, mode; k=10, dataset=train_dataset)
    if (mode=="interpolation")
        return cross_val(featurizer, loss, regularizer, stepsize, error_metric; k=k, dataset=dataset)
    else
        return extrapolation_val(featurizer, loss, regularizer, stepsize, error_metric)
    end
end

# For each model in models, do k-fold cross validation and calculate the average error_metric
# on the val set accross the k-folds.
# Each model in model is a tuple of the form (featurizer, loss, regularizer, stepsize),
# where a featurizer is a funciton that takes in a dataset and returns X,y. 
# Returns errors for each model, and the index of the best model
function test_models(models, error_metric,mode;k=10, dataset=train_dataset)
    errors = []
    for model in tqdm(models)
        error = train_and_validate(model...,error_metric,mode;k=k,dataset=dataset)
        errors = [errors; error]
    end
    i = argmin(errors)
    println("The best model is model ",i)
    return errors,i
end

test_models (generic function with 1 method)

# Feature Engineering and Selection

In [51]:
cat_labels = [:carrier_lg, :carrier_low, :airportid_1, :airportid_2]
cat_labels_sub = [:carrier_lg, :carrier_low]

#Sets of all categories in a particular column
cats_sets = [unique(dataset[:, label]) for label in cat_labels]
cats_sets_sub = [unique(dataset[:, label]) for label in cat_labels_sub]

"Computes a onehot vector for every entry in column given a set of categories cats"
function onehot(column, cats=unique(column))
    result = zeros(length(column), length(cats))
    for i = 1:length(column)
        for j =1:length(cats)
            if column[i] === cats[j]
                result[i, j] = 1
            end    
        end
    end
    result
end

onehot

In [52]:
labels_real = [
    :when, 
    :citymarketid_1, 
    :citymarketid_2,
    :airportid_1,
    :airportid_2,
    :nsmiles,
    :passengers,
    :large_ms,
    :fare_lg,
    :lf_ms,
    :fare_low
]
labels_real_sub = [
    :when, 
    :nsmiles,
    :passengers,
    :large_ms,
    :fare_lg,
    :lf_ms,
    :fare_low
]

7-element Array{Symbol,1}:
 :when
 :nsmiles
 :passengers
 :large_ms
 :fare_lg
 :lf_ms
 :fare_low

In [53]:
# Baseline featurizer
function feats_0(dataset)
    X = [dataset[:when] dataset[:airportid_1] dataset[:airportid_2] ones(size(dataset,1))]
    y = dataset[:fare]
    return X,y
end

# Only numeric features (treating airportid and cityid as numeric)
function feats_1(dataset)
    real_vals = convert(Matrix, dataset[labels_real])
    X = hcat(real_vals, ones(size(dataset,1)))
    y = dataset[:fare]
    return X,y
end

# Numeric and categorical combined (treating airportid and cityid as numerical)
function feats_2(dataset)
    real_vals = convert(Matrix, dataset[labels_real])
    cat_vals = hcat([onehot(dataset[:, cat_labels_sub[i]], cats_sets_sub[i]) for i in 1:size(cat_labels_sub, 1)]...)
    X = hcat(cat_vals, real_vals, ones(size(dataset,1)))
    y = dataset[:fare]
    return X,y
end

# Numeric and categorical combined (treating airportid and cityid as categorical)
function feats_3(dataset)
    real_vals = convert(Matrix, dataset[labels_real_sub])
    cat_vals = hcat([onehot(dataset[:, cat_labels[i]], cats_sets[i]) for i in 1:size(cat_labels, 1)]...)
    X = hcat(cat_vals, real_vals, ones(size(dataset,1)))
    y = dataset[:fare]
    return X,y
end

MSE(L1,L2) = sum((L1.-L2).^2) / size(L1,1)

MSE (generic function with 1 method)

In [62]:
models_0 = [(feats_0, 1/n_train*QuadLoss(), 0.25*QuadReg(), .1),
    (feats_0, 1/n_train*QuadLoss(), 0.5 *QuadReg(), .1),
    (feats_0, 1/n_train*QuadLoss(), 0.75*QuadReg(), .1),
    (feats_0, 1/n_train*QuadLoss(), ZeroReg(), .1),
    (feats_0, 1/n_train*QuadLoss(), 0.25*OneReg(), .1),
    (feats_0, 1/n_train*QuadLoss(), 0.5 *OneReg(), .1),
    (feats_0, 1/n_train*QuadLoss(), 0.75*OneReg(), .1),
    (feats_0, 1/n_train*QuadLoss(), NonNegConstraint(), .1),
    (feats_0, 1/n_train*L1Loss(), 0.25*QuadReg(), .1),
    (feats_0, 1/n_train*L1Loss(), 0.5 *QuadReg(), .1),
    (feats_0, 1/n_train*L1Loss(), 0.75*QuadReg(), .1),
    (feats_0, 1/n_train*L1Loss(), ZeroReg(), .1),
    (feats_0, 1/n_train*L1Loss(), 0.25*OneReg(), .1),
    (feats_0, 1/n_train*L1Loss(), 0.5 *OneReg(), .1),
    (feats_0, 1/n_train*L1Loss(), 0.75*OneReg(), .1),
    (feats_0, 1/n_train*L1Loss(), NonNegConstraint(), .1),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.25*QuadReg(), .1),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *QuadReg(), .1),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.75*QuadReg(), .1),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), ZeroReg(), .1),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.25*OneReg(), .1),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *OneReg(), .1),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.75*OneReg(), .1),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), NonNegConstraint(), .1),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.25*QuadReg(), .1),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *QuadReg(), .1),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.75*QuadReg(), .1),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), ZeroReg(), .1),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.25*OneReg(), .1),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *OneReg(), .1),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.75*OneReg(), .1),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), NonNegConstraint(), .1),
    (feats_0, 1/n_train*QuadLoss(), 0.25*QuadReg(), .3),
    (feats_0, 1/n_train*QuadLoss(), 0.5 *QuadReg(), .3),
    (feats_0, 1/n_train*QuadLoss(), 0.75*QuadReg(), .3),
    (feats_0, 1/n_train*QuadLoss(), ZeroReg(), .3),
    (feats_0, 1/n_train*QuadLoss(), 0.25*OneReg(), .3),
    (feats_0, 1/n_train*QuadLoss(), 0.5 *OneReg(), .3),
    (feats_0, 1/n_train*QuadLoss(), 0.75*OneReg(), .3),
    (feats_0, 1/n_train*QuadLoss(), NonNegConstraint(), .3),
    (feats_0, 1/n_train*L1Loss(), 0.25*QuadReg(), .3),
    (feats_0, 1/n_train*L1Loss(), 0.5 *QuadReg(), .3),
    (feats_0, 1/n_train*L1Loss(), 0.75*QuadReg(), .3),
    (feats_0, 1/n_train*L1Loss(), ZeroReg(), .3),
    (feats_0, 1/n_train*L1Loss(), 0.25*OneReg(), .3),
    (feats_0, 1/n_train*L1Loss(), 0.5 *OneReg(), .3),
    (feats_0, 1/n_train*L1Loss(), 0.75*OneReg(), .3),
    (feats_0, 1/n_train*L1Loss(), NonNegConstraint(), .3),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.25*QuadReg(), .3),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *QuadReg(), .3),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.75*QuadReg(), .3),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), ZeroReg(), .3),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.25*OneReg(), .3),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *OneReg(), .3),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.75*OneReg(), .3),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), NonNegConstraint(), .3),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.25*QuadReg(), .3),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *QuadReg(), .3),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.75*QuadReg(), .3),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), ZeroReg(), .3),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.25*OneReg(), .3),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *OneReg(), .3),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.75*OneReg(), .3),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), NonNegConstraint(), .3),
    (feats_0, 1/n_train*QuadLoss(), 0.25*QuadReg(), .5),
    (feats_0, 1/n_train*QuadLoss(), 0.5 *QuadReg(), .5),
    (feats_0, 1/n_train*QuadLoss(), 0.75*QuadReg(), .5),
    (feats_0, 1/n_train*QuadLoss(), ZeroReg(), .5),
    (feats_0, 1/n_train*QuadLoss(), 0.25*OneReg(), .5),
    (feats_0, 1/n_train*QuadLoss(), 0.5 *OneReg(), .5),
    (feats_0, 1/n_train*QuadLoss(), 0.75*OneReg(), .5),
    (feats_0, 1/n_train*QuadLoss(), NonNegConstraint(), .5),
    (feats_0, 1/n_train*L1Loss(), 0.25*QuadReg(), .5),
    (feats_0, 1/n_train*L1Loss(), 0.5 *QuadReg(), .5),
    (feats_0, 1/n_train*L1Loss(), 0.75*QuadReg(), .5),
    (feats_0, 1/n_train*L1Loss(), ZeroReg(), .5),
    (feats_0, 1/n_train*L1Loss(), 0.25*OneReg(), .5),
    (feats_0, 1/n_train*L1Loss(), 0.5 *OneReg(), .5),
    (feats_0, 1/n_train*L1Loss(), 0.75*OneReg(), .5),
    (feats_0, 1/n_train*L1Loss(), NonNegConstraint(), .5),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.25*QuadReg(), .5),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *QuadReg(), .5),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.75*QuadReg(), .5),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), ZeroReg(), .5),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.25*OneReg(), .5),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *OneReg(), .5),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.75*OneReg(), .5),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), NonNegConstraint(), .5),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.25*QuadReg(), .5),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *QuadReg(), .5),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.75*QuadReg(), .5),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), ZeroReg(), .5),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.25*OneReg(), .5),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *OneReg(), .5),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.75*OneReg(), .5),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), NonNegConstraint(), .5),
    (feats_0, 1/n_train*QuadLoss(), 0.25*QuadReg(), .7),
    (feats_0, 1/n_train*QuadLoss(), 0.5 *QuadReg(), .7),
    (feats_0, 1/n_train*QuadLoss(), 0.75*QuadReg(), .7),
    (feats_0, 1/n_train*QuadLoss(), ZeroReg(), .7),
    (feats_0, 1/n_train*QuadLoss(), 0.25*OneReg(), .7),
    (feats_0, 1/n_train*QuadLoss(), 0.5 *OneReg(), .7),
    (feats_0, 1/n_train*QuadLoss(), 0.75*OneReg(), .7),
    (feats_0, 1/n_train*QuadLoss(), NonNegConstraint(), .7),
    (feats_0, 1/n_train*L1Loss(), 0.25*QuadReg(), .7),
    (feats_0, 1/n_train*L1Loss(), 0.5 *QuadReg(), .7),
    (feats_0, 1/n_train*L1Loss(), 0.75*QuadReg(), .7),
    (feats_0, 1/n_train*L1Loss(), ZeroReg(), .7),
    (feats_0, 1/n_train*L1Loss(), 0.25*OneReg(), .7),
    (feats_0, 1/n_train*L1Loss(), 0.5 *OneReg(), .7),
    (feats_0, 1/n_train*L1Loss(), 0.75*OneReg(), .7),
    (feats_0, 1/n_train*L1Loss(), NonNegConstraint(), .7),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.25*QuadReg(), .7),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *QuadReg(), .7),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.75*QuadReg(), .7),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), ZeroReg(), .7),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.25*OneReg(), .7),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *OneReg(), .7),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.75*OneReg(), .7),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), NonNegConstraint(), .7),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.25*QuadReg(), .7),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *QuadReg(), .7),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.75*QuadReg(), .7),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), ZeroReg(), .7),           # New Best model
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.25*OneReg(), .7),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *OneReg(), .7),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.75*OneReg(), .7),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), NonNegConstraint(), .7),
    (feats_0, 1/n_train*QuadLoss(), 0.25*QuadReg(), .9),
    (feats_0, 1/n_train*QuadLoss(), 0.5 *QuadReg(), .9),
    (feats_0, 1/n_train*QuadLoss(), 0.75*QuadReg(), .9),
    (feats_0, 1/n_train*QuadLoss(), ZeroReg(), .9),
    (feats_0, 1/n_train*QuadLoss(), 0.25*OneReg(), .9),
    (feats_0, 1/n_train*QuadLoss(), 0.5 *OneReg(), .9), 
    (feats_0, 1/n_train*QuadLoss(), 0.75*OneReg(), .9),
    (feats_0, 1/n_train*QuadLoss(), NonNegConstraint(), .9),
    (feats_0, 1/n_train*L1Loss(), 0.25*QuadReg(), .9),
    (feats_0, 1/n_train*L1Loss(), 0.5 *QuadReg(), .9),
    (feats_0, 1/n_train*L1Loss(), 0.75*QuadReg(), .9),
    (feats_0, 1/n_train*L1Loss(), ZeroReg(), .9),
    (feats_0, 1/n_train*L1Loss(), 0.25*OneReg(), .9),
    (feats_0, 1/n_train*L1Loss(), 0.5 *OneReg(), .9),
    (feats_0, 1/n_train*L1Loss(), 0.75*OneReg(), .9),
    (feats_0, 1/n_train*L1Loss(), NonNegConstraint(), .9),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.25*QuadReg(), .9),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *QuadReg(), .9),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.75*QuadReg(), .9),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), ZeroReg(), .9),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.25*OneReg(), .9),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *OneReg(), .9),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.75*OneReg(), .9),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), NonNegConstraint(), .9),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.25*QuadReg(), .9),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *QuadReg(), .9),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.75*QuadReg(), .9),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), ZeroReg(), .9),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.25*OneReg(), .9),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *OneReg(), .9),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.75*OneReg(), .9), # best model 
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), NonNegConstraint(), .9)
]
errors,i = test_models(models_0,MSE,"extrapolation")
print(models_0[i])

100.0%┣████████████████████████████████████████┫ 160/160 [03:48<00:00, 0.7 it/s]
The best model is model 124
(feats_0, QuantileLoss(7.147451933385748e-6, RealDomain(), 0.75), ZeroReg(), 0.7)

In [63]:
models_1 = [
    (feats_1, 1/n_train*QuadLoss(), 0.25*QuadReg(), .1),	
    (feats_1, 1/n_train*QuadLoss(), 0.5 *QuadReg(), .1),
    (feats_1, 1/n_train*QuadLoss(), 0.75*QuadReg(), .1),
    (feats_1, 1/n_train*QuadLoss(), ZeroReg(), .1),
    (feats_1, 1/n_train*QuadLoss(), 0.25*OneReg(), .1),
    (feats_1, 1/n_train*QuadLoss(), 0.5 *OneReg(), .1),
    (feats_1, 1/n_train*QuadLoss(), 0.75*OneReg(), .1),
    (feats_1, 1/n_train*QuadLoss(), NonNegConstraint(), .1),
    (feats_1, 1/n_train*L1Loss(), 0.25*QuadReg(), .1),
    (feats_1, 1/n_train*L1Loss(), 0.5 *QuadReg(), .1),
    (feats_1, 1/n_train*L1Loss(), 0.75*QuadReg(), .1),
    (feats_1, 1/n_train*L1Loss(), ZeroReg(), .1),
    (feats_1, 1/n_train*L1Loss(), 0.25*OneReg(), .1),
    (feats_1, 1/n_train*L1Loss(), 0.5 *OneReg(), .1),
    (feats_1, 1/n_train*L1Loss(), 0.75*OneReg(), .1),
    (feats_1, 1/n_train*L1Loss(), NonNegConstraint(), .1),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.25*QuadReg(), .1),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *QuadReg(), .1),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.75*QuadReg(), .1),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), ZeroReg(), .1),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.25*OneReg(), .1),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *OneReg(), .1),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.75*OneReg(), .1),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), NonNegConstraint(), .1),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.25*QuadReg(), .1),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *QuadReg(), .1),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.75*QuadReg(), .1),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), ZeroReg(), .1),                  # NEW BEST MODEL
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.25*OneReg(), .1),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *OneReg(), .1),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.75*OneReg(), .1),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), NonNegConstraint(), .1),
    (feats_1, 1/n_train*QuadLoss(), 0.25*QuadReg(), .3),
    (feats_1, 1/n_train*QuadLoss(), 0.5 *QuadReg(), .3),
    (feats_1, 1/n_train*QuadLoss(), 0.75*QuadReg(), .3),
    (feats_1, 1/n_train*QuadLoss(), ZeroReg(), .3),
    (feats_1, 1/n_train*QuadLoss(), 0.25*OneReg(), .3),
    (feats_1, 1/n_train*QuadLoss(), 0.5 *OneReg(), .3),
    (feats_1, 1/n_train*QuadLoss(), 0.75*OneReg(), .3),
    (feats_1, 1/n_train*QuadLoss(), NonNegConstraint(), .3),
    (feats_1, 1/n_train*L1Loss(), 0.25*QuadReg(), .3),
    (feats_1, 1/n_train*L1Loss(), 0.5 *QuadReg(), .3),
    (feats_1, 1/n_train*L1Loss(), 0.75*QuadReg(), .3),
    (feats_1, 1/n_train*L1Loss(), ZeroReg(), .3),
    (feats_1, 1/n_train*L1Loss(), 0.25*OneReg(), .3),
    (feats_1, 1/n_train*L1Loss(), 0.5 *OneReg(), .3),
    (feats_1, 1/n_train*L1Loss(), 0.75*OneReg(), .3),
    (feats_1, 1/n_train*L1Loss(), NonNegConstraint(), .3),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.25*QuadReg(), .3),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *QuadReg(), .3),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.75*QuadReg(), .3),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), ZeroReg(), .3),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.25*OneReg(), .3),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *OneReg(), .3),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.75*OneReg(), .3),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), NonNegConstraint(), .3),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.25*QuadReg(), .3),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *QuadReg(), .3),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.75*QuadReg(), .3),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), ZeroReg(), .3),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.25*OneReg(), .3),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *OneReg(), .3),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.75*OneReg(), .3),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), NonNegConstraint(), .3),
    (feats_1, 1/n_train*QuadLoss(), 0.25*QuadReg(), .5),
    (feats_1, 1/n_train*QuadLoss(), 0.5 *QuadReg(), .5),
    (feats_1, 1/n_train*QuadLoss(), 0.75*QuadReg(), .5),
    (feats_1, 1/n_train*QuadLoss(), ZeroReg(), .5),
    (feats_1, 1/n_train*QuadLoss(), 0.25*OneReg(), .5),
    (feats_1, 1/n_train*QuadLoss(), 0.5 *OneReg(), .5),
    (feats_1, 1/n_train*QuadLoss(), 0.75*OneReg(), .5),
    (feats_1, 1/n_train*QuadLoss(), NonNegConstraint(), .5),
    (feats_1, 1/n_train*L1Loss(), 0.25*QuadReg(), .5),
    (feats_1, 1/n_train*L1Loss(), 0.5 *QuadReg(), .5),
    (feats_1, 1/n_train*L1Loss(), 0.75*QuadReg(), .5),
    (feats_1, 1/n_train*L1Loss(), ZeroReg(), .5),
    (feats_1, 1/n_train*L1Loss(), 0.25*OneReg(), .5),
    (feats_1, 1/n_train*L1Loss(), 0.5 *OneReg(), .5),
    (feats_1, 1/n_train*L1Loss(), 0.75*OneReg(), .5),
    (feats_1, 1/n_train*L1Loss(), NonNegConstraint(), .5),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.25*QuadReg(), .5),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *QuadReg(), .5),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.75*QuadReg(), .5),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), ZeroReg(), .5),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.25*OneReg(), .5),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *OneReg(), .5),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.75*OneReg(), .5),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), NonNegConstraint(), .5),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.25*QuadReg(), .5),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *QuadReg(), .5), # best model
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.75*QuadReg(), .5),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), ZeroReg(), .5),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.25*OneReg(), .5),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *OneReg(), .5),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.75*OneReg(), .5),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), NonNegConstraint(), .5),
    (feats_1, 1/n_train*QuadLoss(), 0.25*QuadReg(), .7),
    (feats_1, 1/n_train*QuadLoss(), 0.5 *QuadReg(), .7),
    (feats_1, 1/n_train*QuadLoss(), 0.75*QuadReg(), .7),
    (feats_1, 1/n_train*QuadLoss(), ZeroReg(), .7),
    (feats_1, 1/n_train*QuadLoss(), 0.25*OneReg(), .7),
    (feats_1, 1/n_train*QuadLoss(), 0.5 *OneReg(), .7),
    (feats_1, 1/n_train*QuadLoss(), 0.75*OneReg(), .7),
    (feats_1, 1/n_train*QuadLoss(), NonNegConstraint(), .7),
    (feats_1, 1/n_train*L1Loss(), 0.25*QuadReg(), .7),
    (feats_1, 1/n_train*L1Loss(), 0.5 *QuadReg(), .7),
    (feats_1, 1/n_train*L1Loss(), 0.75*QuadReg(), .7),
    (feats_1, 1/n_train*L1Loss(), ZeroReg(), .7),
    (feats_1, 1/n_train*L1Loss(), 0.25*OneReg(), .7),
    (feats_1, 1/n_train*L1Loss(), 0.5 *OneReg(), .7),
    (feats_1, 1/n_train*L1Loss(), 0.75*OneReg(), .7),
    (feats_1, 1/n_train*L1Loss(), NonNegConstraint(), .7),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.25*QuadReg(), .7),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *QuadReg(), .7),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.75*QuadReg(), .7),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), ZeroReg(), .7),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.25*OneReg(), .7),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *OneReg(), .7),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.75*OneReg(), .7),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), NonNegConstraint(), .7),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.25*QuadReg(), .7),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *QuadReg(), .7),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.75*QuadReg(), .7),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), ZeroReg(), .7),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.25*OneReg(), .7),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *OneReg(), .7),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.75*OneReg(), .7),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), NonNegConstraint(), .7),
    (feats_1, 1/n_train*QuadLoss(), 0.25*QuadReg(), .9),
    (feats_1, 1/n_train*QuadLoss(), 0.5 *QuadReg(), .9),
    (feats_1, 1/n_train*QuadLoss(), 0.75*QuadReg(), .9),
    (feats_1, 1/n_train*QuadLoss(), ZeroReg(), .9),
    (feats_1, 1/n_train*QuadLoss(), 0.25*OneReg(), .9),
    (feats_1, 1/n_train*QuadLoss(), 0.5 *OneReg(), .9), 
    (feats_1, 1/n_train*QuadLoss(), 0.75*OneReg(), .9),
    (feats_1, 1/n_train*QuadLoss(), NonNegConstraint(), .9),
    (feats_1, 1/n_train*L1Loss(), 0.25*QuadReg(), .9),
    (feats_1, 1/n_train*L1Loss(), 0.5 *QuadReg(), .9),
    (feats_1, 1/n_train*L1Loss(), 0.75*QuadReg(), .9),
    (feats_1, 1/n_train*L1Loss(), ZeroReg(), .9),
    (feats_1, 1/n_train*L1Loss(), 0.25*OneReg(), .9),
    (feats_1, 1/n_train*L1Loss(), 0.5 *OneReg(), .9),
    (feats_1, 1/n_train*L1Loss(), 0.75*OneReg(), .9),
    (feats_1, 1/n_train*L1Loss(), NonNegConstraint(), .9),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.25*QuadReg(), .9),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *QuadReg(), .9),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.75*QuadReg(), .9),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), ZeroReg(), .9),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.25*OneReg(), .9),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *OneReg(), .9),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.75*OneReg(), .9),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), NonNegConstraint(), .9),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.25*QuadReg(), .9),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *QuadReg(), .9),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.75*QuadReg(), .9),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), ZeroReg(), .9),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.25*OneReg(), .9),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *OneReg(), .9),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.75*OneReg(), .9),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), NonNegConstraint(), .9)
]

errors,i = test_models(models_1,MSE,"extrapolation")


100.0%┣████████████████████████████████████████┫ 160/160 [03:29<00:00, 0.8 it/s]
The best model is model 28
(feats_0, QuadLoss(7.147451933385748e-6, RealDomain()), QuadReg(0.25), 0.1)

In [66]:
models_1[28]

(feats_1, QuantileLoss(7.147451933385748e-6, RealDomain(), 0.75), ZeroReg(), 0.1)

In [64]:
models_2 = [
    (feats_2, 1/n_train*QuadLoss(), 0.25*QuadReg(), .1),	
    (feats_2, 1/n_train*QuadLoss(), 0.5 *QuadReg(), .1),
    (feats_2, 1/n_train*QuadLoss(), 0.75*QuadReg(), .1),
    (feats_2, 1/n_train*QuadLoss(), ZeroReg(), .1),
    (feats_2, 1/n_train*QuadLoss(), 0.25*OneReg(), .1),
    (feats_2, 1/n_train*QuadLoss(), 0.5 *OneReg(), .1),
    (feats_2, 1/n_train*QuadLoss(), 0.75*OneReg(), .1),
    (feats_2, 1/n_train*QuadLoss(), NonNegConstraint(), .1),
    (feats_2, 1/n_train*L1Loss(), 0.25*QuadReg(), .1),
    (feats_2, 1/n_train*L1Loss(), 0.5 *QuadReg(), .1),
    (feats_2, 1/n_train*L1Loss(), 0.75*QuadReg(), .1),
    (feats_2, 1/n_train*L1Loss(), ZeroReg(), .1),
    (feats_2, 1/n_train*L1Loss(), 0.25*OneReg(), .1),
    (feats_2, 1/n_train*L1Loss(), 0.5 *OneReg(), .1),
    (feats_2, 1/n_train*L1Loss(), 0.75*OneReg(), .1),
    (feats_2, 1/n_train*L1Loss(), NonNegConstraint(), .1),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.25*QuadReg(), .1),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *QuadReg(), .1),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.75*QuadReg(), .1),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), ZeroReg(), .1),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.25*OneReg(), .1),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *OneReg(), .1),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.75*OneReg(), .1),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), NonNegConstraint(), .1),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.25*QuadReg(), .1),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *QuadReg(), .1),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.75*QuadReg(), .1),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), ZeroReg(), .1),               #NEW BEST MODEL
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.25*OneReg(), .1),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *OneReg(), .1),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.75*OneReg(), .1),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), NonNegConstraint(), .1),
    (feats_2, 1/n_train*QuadLoss(), 0.25*QuadReg(), .3),
    (feats_2, 1/n_train*QuadLoss(), 0.5 *QuadReg(), .3),
    (feats_2, 1/n_train*QuadLoss(), 0.75*QuadReg(), .3),
    (feats_2, 1/n_train*QuadLoss(), ZeroReg(), .3),
    (feats_2, 1/n_train*QuadLoss(), 0.25*OneReg(), .3),
    (feats_2, 1/n_train*QuadLoss(), 0.5 *OneReg(), .3),
    (feats_2, 1/n_train*QuadLoss(), 0.75*OneReg(), .3),
    (feats_2, 1/n_train*QuadLoss(), NonNegConstraint(), .3),
    (feats_2, 1/n_train*L1Loss(), 0.25*QuadReg(), .3),
    (feats_2, 1/n_train*L1Loss(), 0.5 *QuadReg(), .3),
    (feats_2, 1/n_train*L1Loss(), 0.75*QuadReg(), .3),
    (feats_2, 1/n_train*L1Loss(), ZeroReg(), .3),
    (feats_2, 1/n_train*L1Loss(), 0.25*OneReg(), .3),
    (feats_2, 1/n_train*L1Loss(), 0.5 *OneReg(), .3),
    (feats_2, 1/n_train*L1Loss(), 0.75*OneReg(), .3),
    (feats_2, 1/n_train*L1Loss(), NonNegConstraint(), .3),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.25*QuadReg(), .3),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *QuadReg(), .3),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.75*QuadReg(), .3),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), ZeroReg(), .3),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.25*OneReg(), .3),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *OneReg(), .3),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.75*OneReg(), .3),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), NonNegConstraint(), .3),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.25*QuadReg(), .3),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *QuadReg(), .3),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.75*QuadReg(), .3),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), ZeroReg(), .3),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.25*OneReg(), .3),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *OneReg(), .3),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.75*OneReg(), .3),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), NonNegConstraint(), .3),
    (feats_2, 1/n_train*QuadLoss(), 0.25*QuadReg(), .5),
    (feats_2, 1/n_train*QuadLoss(), 0.5 *QuadReg(), .5),
    (feats_2, 1/n_train*QuadLoss(), 0.75*QuadReg(), .5),
    (feats_2, 1/n_train*QuadLoss(), ZeroReg(), .5),
    (feats_2, 1/n_train*QuadLoss(), 0.25*OneReg(), .5),
    (feats_2, 1/n_train*QuadLoss(), 0.5 *OneReg(), .5),
    (feats_2, 1/n_train*QuadLoss(), 0.75*OneReg(), .5),
    (feats_2, 1/n_train*QuadLoss(), NonNegConstraint(), .5),
    (feats_2, 1/n_train*L1Loss(), 0.25*QuadReg(), .5),
    (feats_2, 1/n_train*L1Loss(), 0.5 *QuadReg(), .5),
    (feats_2, 1/n_train*L1Loss(), 0.75*QuadReg(), .5),
    (feats_2, 1/n_train*L1Loss(), ZeroReg(), .5),
    (feats_2, 1/n_train*L1Loss(), 0.25*OneReg(), .5),
    (feats_2, 1/n_train*L1Loss(), 0.5 *OneReg(), .5),
    (feats_2, 1/n_train*L1Loss(), 0.75*OneReg(), .5),
    (feats_2, 1/n_train*L1Loss(), NonNegConstraint(), .5),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.25*QuadReg(), .5),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *QuadReg(), .5),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.75*QuadReg(), .5),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), ZeroReg(), .5),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.25*OneReg(), .5),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *OneReg(), .5),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.75*OneReg(), .5),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), NonNegConstraint(), .5),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.25*QuadReg(), .5),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *QuadReg(), .5), # best model
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.75*QuadReg(), .5),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), ZeroReg(), .5),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.25*OneReg(), .5),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *OneReg(), .5),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.75*OneReg(), .5),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), NonNegConstraint(), .5),
    (feats_2, 1/n_train*QuadLoss(), 0.25*QuadReg(), .7),
    (feats_2, 1/n_train*QuadLoss(), 0.5 *QuadReg(), .7),
    (feats_2, 1/n_train*QuadLoss(), 0.75*QuadReg(), .7),
    (feats_2, 1/n_train*QuadLoss(), ZeroReg(), .7),
    (feats_2, 1/n_train*QuadLoss(), 0.25*OneReg(), .7),
    (feats_2, 1/n_train*QuadLoss(), 0.5 *OneReg(), .7),
    (feats_2, 1/n_train*QuadLoss(), 0.75*OneReg(), .7),
    (feats_2, 1/n_train*QuadLoss(), NonNegConstraint(), .7),
    (feats_2, 1/n_train*L1Loss(), 0.25*QuadReg(), .7),
    (feats_2, 1/n_train*L1Loss(), 0.5 *QuadReg(), .7),
    (feats_2, 1/n_train*L1Loss(), 0.75*QuadReg(), .7),
    (feats_2, 1/n_train*L1Loss(), ZeroReg(), .7),
    (feats_2, 1/n_train*L1Loss(), 0.25*OneReg(), .7),
    (feats_2, 1/n_train*L1Loss(), 0.5 *OneReg(), .7),
    (feats_2, 1/n_train*L1Loss(), 0.75*OneReg(), .7),
    (feats_2, 1/n_train*L1Loss(), NonNegConstraint(), .7),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.25*QuadReg(), .7),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *QuadReg(), .7),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.75*QuadReg(), .7),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), ZeroReg(), .7),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.25*OneReg(), .7),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *OneReg(), .7),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.75*OneReg(), .7),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), NonNegConstraint(), .7),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.25*QuadReg(), .7),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *QuadReg(), .7),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.75*QuadReg(), .7),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), ZeroReg(), .7),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.25*OneReg(), .7),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *OneReg(), .7),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.75*OneReg(), .7),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), NonNegConstraint(), .7),
    (feats_2, 1/n_train*QuadLoss(), 0.25*QuadReg(), .9),
    (feats_2, 1/n_train*QuadLoss(), 0.5 *QuadReg(), .9),
    (feats_2, 1/n_train*QuadLoss(), 0.75*QuadReg(), .9),
    (feats_2, 1/n_train*QuadLoss(), ZeroReg(), .9),
    (feats_2, 1/n_train*QuadLoss(), 0.25*OneReg(), .9),
    (feats_2, 1/n_train*QuadLoss(), 0.5 *OneReg(), .9), 
    (feats_2, 1/n_train*QuadLoss(), 0.75*OneReg(), .9),
    (feats_2, 1/n_train*QuadLoss(), NonNegConstraint(), .9),
    (feats_2, 1/n_train*L1Loss(), 0.25*QuadReg(), .9),
    (feats_2, 1/n_train*L1Loss(), 0.5 *QuadReg(), .9),
    (feats_2, 1/n_train*L1Loss(), 0.75*QuadReg(), .9),
    (feats_2, 1/n_train*L1Loss(), ZeroReg(), .9),
    (feats_2, 1/n_train*L1Loss(), 0.25*OneReg(), .9),
    (feats_2, 1/n_train*L1Loss(), 0.5 *OneReg(), .9),
    (feats_2, 1/n_train*L1Loss(), 0.75*OneReg(), .9),
    (feats_2, 1/n_train*L1Loss(), NonNegConstraint(), .9),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.25*QuadReg(), .9),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *QuadReg(), .9),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.75*QuadReg(), .9),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), ZeroReg(), .9),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.25*OneReg(), .9),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *OneReg(), .9),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.75*OneReg(), .9),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), NonNegConstraint(), .9),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.25*QuadReg(), .9),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *QuadReg(), .9),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.75*QuadReg(), .9),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), ZeroReg(), .9),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.25*OneReg(), .9),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *OneReg(), .9),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.75*OneReg(), .9),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), NonNegConstraint(), .9)
]

errors,i = test_models(models_2,MSE,"extrapolation")
print(models_2[i])

100.0%┣████████████████████████████████████████┫ 160/160 [16:38<00:00, 0.2 it/s]
The best model is model 28
(feats_2, QuantileLoss(7.147451933385748e-6, RealDomain(), 0.75), ZeroReg(), 0.1)

In [65]:
models_3 = [
    (feats_3, 1/n_train*QuadLoss(), 0.25*QuadReg(), .1),	
    (feats_3, 1/n_train*QuadLoss(), 0.5 *QuadReg(), .1),
    (feats_3, 1/n_train*QuadLoss(), 0.75*QuadReg(), .1),
    (feats_3, 1/n_train*QuadLoss(), ZeroReg(), .1),
    (feats_3, 1/n_train*QuadLoss(), 0.25*OneReg(), .1),
    (feats_3, 1/n_train*QuadLoss(), 0.5 *OneReg(), .1),
    (feats_3, 1/n_train*QuadLoss(), 0.75*OneReg(), .1),
    (feats_3, 1/n_train*QuadLoss(), NonNegConstraint(), .1),
    (feats_3, 1/n_train*L1Loss(), 0.25*QuadReg(), .1),
    (feats_3, 1/n_train*L1Loss(), 0.5 *QuadReg(), .1),
    (feats_3, 1/n_train*L1Loss(), 0.75*QuadReg(), .1),
    (feats_3, 1/n_train*L1Loss(), ZeroReg(), .1),
    (feats_3, 1/n_train*L1Loss(), 0.25*OneReg(), .1),
    (feats_3, 1/n_train*L1Loss(), 0.5 *OneReg(), .1),
    (feats_3, 1/n_train*L1Loss(), 0.75*OneReg(), .1),
    (feats_3, 1/n_train*L1Loss(), NonNegConstraint(), .1),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.25*QuadReg(), .1),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *QuadReg(), .1),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.75*QuadReg(), .1),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), ZeroReg(), .1),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.25*OneReg(), .1),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *OneReg(), .1),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.75*OneReg(), .1),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), NonNegConstraint(), .1),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.25*QuadReg(), .1),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *QuadReg(), .1),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.75*QuadReg(), .1),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), ZeroReg(), .1),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.25*OneReg(), .1),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *OneReg(), .1),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.75*OneReg(), .1),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), NonNegConstraint(), .1),
    (feats_3, 1/n_train*QuadLoss(), 0.25*QuadReg(), .3),
    (feats_3, 1/n_train*QuadLoss(), 0.5 *QuadReg(), .3),
    (feats_3, 1/n_train*QuadLoss(), 0.75*QuadReg(), .3),
    (feats_3, 1/n_train*QuadLoss(), ZeroReg(), .3),
    (feats_3, 1/n_train*QuadLoss(), 0.25*OneReg(), .3),
    (feats_3, 1/n_train*QuadLoss(), 0.5 *OneReg(), .3),
    (feats_3, 1/n_train*QuadLoss(), 0.75*OneReg(), .3),
    (feats_3, 1/n_train*QuadLoss(), NonNegConstraint(), .3),
    (feats_3, 1/n_train*L1Loss(), 0.25*QuadReg(), .3),
    (feats_3, 1/n_train*L1Loss(), 0.5 *QuadReg(), .3),
    (feats_3, 1/n_train*L1Loss(), 0.75*QuadReg(), .3),
    (feats_3, 1/n_train*L1Loss(), ZeroReg(), .3),
    (feats_3, 1/n_train*L1Loss(), 0.25*OneReg(), .3),
    (feats_3, 1/n_train*L1Loss(), 0.5 *OneReg(), .3),
    (feats_3, 1/n_train*L1Loss(), 0.75*OneReg(), .3),
    (feats_3, 1/n_train*L1Loss(), NonNegConstraint(), .3),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.25*QuadReg(), .3),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *QuadReg(), .3),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.75*QuadReg(), .3),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), ZeroReg(), .3),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.25*OneReg(), .3),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *OneReg(), .3),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.75*OneReg(), .3),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), NonNegConstraint(), .3),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.25*QuadReg(), .3),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *QuadReg(), .3),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.75*QuadReg(), .3),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), ZeroReg(), .3),   # NEW BEST MODEL
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.25*OneReg(), .3),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *OneReg(), .3),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.75*OneReg(), .3),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), NonNegConstraint(), .3),
    (feats_3, 1/n_train*QuadLoss(), 0.25*QuadReg(), .5),
    (feats_3, 1/n_train*QuadLoss(), 0.5 *QuadReg(), .5),
    (feats_3, 1/n_train*QuadLoss(), 0.75*QuadReg(), .5),
    (feats_3, 1/n_train*QuadLoss(), ZeroReg(), .5),
    (feats_3, 1/n_train*QuadLoss(), 0.25*OneReg(), .5),
    (feats_3, 1/n_train*QuadLoss(), 0.5 *OneReg(), .5),
    (feats_3, 1/n_train*QuadLoss(), 0.75*OneReg(), .5),
    (feats_3, 1/n_train*QuadLoss(), NonNegConstraint(), .5),
    (feats_3, 1/n_train*L1Loss(), 0.25*QuadReg(), .5),
    (feats_3, 1/n_train*L1Loss(), 0.5 *QuadReg(), .5),
    (feats_3, 1/n_train*L1Loss(), 0.75*QuadReg(), .5),
    (feats_3, 1/n_train*L1Loss(), ZeroReg(), .5),
    (feats_3, 1/n_train*L1Loss(), 0.25*OneReg(), .5),
    (feats_3, 1/n_train*L1Loss(), 0.5 *OneReg(), .5),
    (feats_3, 1/n_train*L1Loss(), 0.75*OneReg(), .5),
    (feats_3, 1/n_train*L1Loss(), NonNegConstraint(), .5),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.25*QuadReg(), .5),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *QuadReg(), .5),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.75*QuadReg(), .5),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), ZeroReg(), .5),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.25*OneReg(), .5),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *OneReg(), .5),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.75*OneReg(), .5),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), NonNegConstraint(), .5),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.25*QuadReg(), .5),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *QuadReg(), .5),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.75*QuadReg(), .5),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), ZeroReg(), .5),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.25*OneReg(), .5),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *OneReg(), .5),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.75*OneReg(), .5),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), NonNegConstraint(), .5),
    (feats_3, 1/n_train*QuadLoss(), 0.25*QuadReg(), .7),
    (feats_3, 1/n_train*QuadLoss(), 0.5 *QuadReg(), .7),
    (feats_3, 1/n_train*QuadLoss(), 0.75*QuadReg(), .7),
    (feats_3, 1/n_train*QuadLoss(), ZeroReg(), .7),
    (feats_3, 1/n_train*QuadLoss(), 0.25*OneReg(), .7),
    (feats_3, 1/n_train*QuadLoss(), 0.5 *OneReg(), .7),
    (feats_3, 1/n_train*QuadLoss(), 0.75*OneReg(), .7),
    (feats_3, 1/n_train*QuadLoss(), NonNegConstraint(), .7),
    (feats_3, 1/n_train*L1Loss(), 0.25*QuadReg(), .7),
    (feats_3, 1/n_train*L1Loss(), 0.5 *QuadReg(), .7),
    (feats_3, 1/n_train*L1Loss(), 0.75*QuadReg(), .7),
    (feats_3, 1/n_train*L1Loss(), ZeroReg(), .7),
    (feats_3, 1/n_train*L1Loss(), 0.25*OneReg(), .7),
    (feats_3, 1/n_train*L1Loss(), 0.5 *OneReg(), .7),
    (feats_3, 1/n_train*L1Loss(), 0.75*OneReg(), .7),
    (feats_3, 1/n_train*L1Loss(), NonNegConstraint(), .7),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.25*QuadReg(), .7),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *QuadReg(), .7),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.75*QuadReg(), .7),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), ZeroReg(), .7),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.25*OneReg(), .7),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *OneReg(), .7),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.75*OneReg(), .7),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), NonNegConstraint(), .7),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.25*QuadReg(), .7),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *QuadReg(), .7),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.75*QuadReg(), .7),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), ZeroReg(), .7), # best model
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.25*OneReg(), .7),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *OneReg(), .7),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.75*OneReg(), .7),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), NonNegConstraint(), .7),
    (feats_3, 1/n_train*QuadLoss(), 0.25*QuadReg(), .9),
    (feats_3, 1/n_train*QuadLoss(), 0.5 *QuadReg(), .9),
    (feats_3, 1/n_train*QuadLoss(), 0.75*QuadReg(), .9),
    (feats_3, 1/n_train*QuadLoss(), ZeroReg(), .9),
    (feats_3, 1/n_train*QuadLoss(), 0.25*OneReg(), .9),
    (feats_3, 1/n_train*QuadLoss(), 0.5 *OneReg(), .9), 
    (feats_3, 1/n_train*QuadLoss(), 0.75*OneReg(), .9),
    (feats_3, 1/n_train*QuadLoss(), NonNegConstraint(), .9),
    (feats_3, 1/n_train*L1Loss(), 0.25*QuadReg(), .9),
    (feats_3, 1/n_train*L1Loss(), 0.5 *QuadReg(), .9),
    (feats_3, 1/n_train*L1Loss(), 0.75*QuadReg(), .9),
    (feats_3, 1/n_train*L1Loss(), ZeroReg(), .9),
    (feats_3, 1/n_train*L1Loss(), 0.25*OneReg(), .9),
    (feats_3, 1/n_train*L1Loss(), 0.5 *OneReg(), .9),
    (feats_3, 1/n_train*L1Loss(), 0.75*OneReg(), .9),
    (feats_3, 1/n_train*L1Loss(), NonNegConstraint(), .9),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.25*QuadReg(), .9),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *QuadReg(), .9),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.75*QuadReg(), .9),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), ZeroReg(), .9),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.25*OneReg(), .9),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *OneReg(), .9),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.75*OneReg(), .9),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), NonNegConstraint(), .9),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.25*QuadReg(), .9),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *QuadReg(), .9),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.75*QuadReg(), .9),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), ZeroReg(), .9),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.25*OneReg(), .9),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *OneReg(), .9),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.75*OneReg(), .9),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), NonNegConstraint(), .9)
]

errors,i = test_models(models_3,MSE,"extrapolation")
print(models_3[i])

100.0%┣████████████████████████████████████████┫ 160/160 [44:35<00:00, 0.1 it/s]
The best model is model 60
(feats_3, QuantileLoss(7.147451933385748e-6, RealDomain(), 0.75), ZeroReg(), 0.3)